###### This is pre-release documentation, but please **[log issues](https://github.com/InstituteforDiseaseModeling/jupyter-notebooks-comps/issues)** found.

# COMPS and Jupyter `v0.0.1`

Demonstrations of transactions between **Jupyter Notebooks** and IDM's **Computational Modeling Platform Service** (COMPS).

COMPS provides for the submission, execution, and management of computational simulations on high-performance computing clusters (HPC). COMPS employs a scaleable RESTful service architecture, including a internet accessible website, [comps.idmod.org](https://comps.idmod.org).

<blockquote style="background:lightpink;margin:0;padding:1rem"><b>NOTE: This Notebook currently only works in a local environment with version v3.7.0 of COMPS (yet to be released).</b></blockquote>

There are at least two ways to interface with COMPS from a Notebook:
- With JavaScript interacting with the website itself.
- With Python via the [PyCOMPS](https://github.com/InstituteforDiseaseModeling/pyCOMPS) library.

The first to be addressed here is the JavaScript method. These interactions can be made from a Jupyter Notebook running in its customary browser-based read-eval-print loop environment (REPL). Since the COMPS UI is browser-based too, there is some degree of interaction that is possible between them, and this is best achieved using JavaScript.

The following commands with load and then test [NotebookJS](https://github.com/jorgehpo/notebookJS) a popular Python library for executing JavaScript commands. Execute these now so that the library is available for subsequent code...

In [ ]:
from notebookjs import execute_js

In [ ]:
hello_js = """
function hello(selector, data){
    document.querySelector(selector).textContent=data.text;
}
"""

In [ ]:
execute_js(hello_js, "hello", {"text": "Hello NotebookJS!"})

Quite a lot of JavaScript will be required to interact between two windows of a browser, from Python expressions to JavaScript, so [a custom library](https://github.com/ptrdo/hello-comps/blob/main/hello_comps.js) of JavaScript will be loaded here into memory to facilitate this communication while keeping the Notebook relatively simple...

In [ ]:
with open("./hello_comps.js", "r") as f:
    comps_lib = f.read()

The following commands will call the NotebookJS library's `execute_js` command and pass the arguments to the custom JavaScript code. The first command will see if COMPS is open in another browser tab, and if not, will open one...

In [ ]:
execute_js(comps_lib, "comps", {"test":"window"})

In [ ]:
execute_js(comps_lib, "comps", {"test":"cookies"})

In [ ]:
execute_js(comps_lib, "comps", {"test":"signin","username":"psylwester","password":"groot"})

In [ ]:
execute_js(comps_lib, "comps", {"test":"signin","username":"psylwester","password":"groot"})

In [ ]:
execute_js(comps_lib, "comps", {"test":"navigate"});

In [ ]:
# C:\[path\for\package\source]
# git clone https://github.com/InstituteforDiseaseModeling/pyCOMPS.git
# cd pyCOMPS
# python setup.py install

In [ ]:
# pip -V
# C:\Users\[username]\AppData\Local\Programs\Python\Python39\lib\site-packages\

In [ ]:
# import os
# import sys
# sys.path.insert(0,"C:\\path\\to\\egg")

In [ ]:
from COMPS import Client
from COMPS import AuthManager, CredentialPrompt
from COMPS.Data import QueryCriteria, Experiment
from COMPS.utils.get_output_files_for_experiment import get_files

compshost = 'https://comps2.idmod.org'
if __name__ == '__main__':
    Client.login(compshost)
    last_exp = Experiment.get(query_criteria=QueryCriteria().where('owner=psylwester').orderby('date_created desc').count(1))[0]
    print(last_exp)
    get_files(last_exp.id, ['stdout.txt'])